# Laboratoire 4 - Cinématique

> **Attention!** Il est *strictement interdit* de s'approcher des commandes qui font avancer le robot si il y a le moindre risque que le robot ne tombe par terre. Les équipes qui ont un accidents de robot devront chanter *Never Gonna Give You Up* devant toute la classe.

Au cours de ce laboratoire nous allons étudier la façon dont le robot se déplace. Comme vous le constaterez, le sujet est plus complexe qu'il n'y paraît! Connectons-nous d'abord au robot.


In [16]:
from robmob import robot, sensors
from time import sleep
from math import pi as PI

import numpy as np

robot_ip = '192.168.0.109'
robot = robot.Robot(robot_ip)
robot.connect()

## Partie 1 - Le problème du contrôle

Pour cette première expérience, utilisez les fonctions de la librairie `robmob` pour ordonner au robot d'avancer de un mètre.

In [ ]:
robot.linear_movement(0.2, 5)  # Ordonne au robot d'avancer à 0.2 m/s pendant 5 s

Vous constaterez que les kobukis se déplacent de beaucoup moins qu'un mètre avec l'exécution de cette command. Qu'est-ce qui se passe? Pour le comprendre, il faut d'abord se demander comment le robot exécute notre demande.

Nous envoyons une commande de vitesse au robot. Le robot active ses moteurs, puis après un certain $\Delta t$ le robot consulte les encodeurs de ses roues pour valider sa vitesse. La différence entre la vitesse demandée et la vitesse réelle constitue une erreur. Cette erreur est fournie à une fonction appelée un [correcteur PID](https://fr.wikipedia.org/wiki/R%C3%A9gulateur_PID) qui traduit l'erreur en une nouvelle consigne. Cette nouvelle consigne tente de corriger l'erreur sur la vitesse du robot, pour exécuter aussi bien que possible la commande qu'on lui a demandé.

Malheureusement, il est difficile de créer un correcteur parfait. Dans le cas qui nous concerne, le correcteur semble avoir de la difficulté à atteindre la vitesse demandée, surtout au début de la trajectoire. Voici un graphique de la vitesse instantanée d'un robot kobuki par rapport au temps. La vitesse demandée est en rouge. L'apparence de ce graphique explique pourquoi les distances parcourues par le robot sont plus courtes que les distances demandées.

![Vitesse de la kobuki par rapport au temps](img/vitesse_kobuki.png)



## Partie 2 - Rotation

Dans cette section nous effectuerons des rotations et nous validerons comment le robot peut déduire son état à l'aide de ses différents senseurs.


En premier lieu, fixez une règle sur le robot et placez une feuille de papier de façon à pouvoir facilement calculer des angles, comme sur l'image.

![Montage rotation](img/montage_rotation.png)

Maintenant demandons au robot de faire une rotation de $\pi$ radians. 

In [ ]:
robot.angular_movement(PI / 4, 4) # Demande au robot de tourner à PI/4 radians/seconde pendant 4 secondes.

Mesurez la rotation que le robot a effectué. Vous constaterez que la rotation résultante est beaucoup plus faible que la rotation demandée.  Mais le robot en est-il conscient?

### Odométrie

Nous allons utiliser l'odomètre du robot pour estimer sa rotation réelle. Pour ce faire nous pouvons ajouter un `OdometerTicksSensor` à notre objet `robot`.

In [2]:
odometer = sensors.OdometerTicksSensor()
robot.add_sensor(odometer)
sleep(1)
odometer.peek_data()

(1476306586.6465306, 54851, 10803)

L'odomètre nous renvoie trois valeurs: le *timestamp* du message, le nombre de ticks enregistrés pour la roue gauche, et le nombre de ticks enregistrés par la roue droite. Il est possible de convertir ces ticks en distance parcourue! Sachant que la variable `odometer.TICK_TO_METER` indique le déplacement d'une roue lorsqu'elle effectue un tick, le déplacement de la roue gauche depuis que le robot est activé est comme suit.

In [ ]:
message = odometer.peek_data()
displacement = message[1] * odometer.TICK_TO_METER

Le code suivant demande au robot de faire une rotation de $\pi$ radians, et enregistre le status de l'odomètre avant et après le déplacement. Sachant cela, calculez le déplacement subi par la roue gauche et la roue droite.

In [3]:
odometer_before = odometer.peek_data()
robot.angular_movement(PI / 4, 4)
sleep(1)
odometer_after = odometer.peek_data()

In [4]:
displacement_right = (odometer_after[2] - odometer_before[2]) * odometer.TICK_TO_METER
displacement_left = (odometer_after[1] - odometer_before[1]) * odometer.TICK_TO_METER

print('Left (m): {}, Right (m): {}'.format(displacement_left, displacement_right))

Left (m): -0.23702671949321266, Right (m): 0.2416324923800905


À l'aide de ces déplacements, calculez la rotation que le robot a effectué, du point de vue de l'odomètre. La valeur `robot.DISTANCE_CENTER_TO_WHEEL` pourrait vous être utile.

In [10]:
rotation = (displacement_right - displacement_left) / (2*robot.DISTANCE_CENTER_TO_WHEEL)

print('Rotation (rad): {}, Rotation (deg): {}'.format(rotation, rotation / (2*PI) * 360))

Rotation (rad): 2.1757236903331965, Rotation (deg): 124.65978484272064


### Gyroscope

Vous êtes déjà à l'aise avec le gyroscope. Nous allons intégrer ses mesures pour faire un nouvel estimé de la rotation. Mais d'abord, trouvons rapidement un biais. Pour ce faire, laissez le robot immobile pendant que vous exécutez le code suivant.

In [27]:
gyro = sensors.GyroSensor(6 * sensors.GyroSensor.SAMPLE_RATE) # Le buffer a une durée de 6 secondes.
robot.add_sensor(gyro)

sleep(7)

gyro_data = gyro.read_buffer() # La commande read_buffer capture tout le contenu du buffer. Le buffer est vide après.
gyro_data = np.array([x[2] for x in data]) # On a seulement besoin de la vitesse angulaire autour de l'axe z.
bias = np.mean(gyro_data)

print('Bias (deg/s): {}'.format(bias))

Bias (rad/s): -0.9941679566563468


Avec un peu de chance, le routeur va bien se comporter et vous recevez les mesures du gyro en temps réel. Si c'est le cas, exécutez du code de façon à intégrer le déplacement angulaire du robot alors qu'il exécute la même commande.

In [32]:
robot.angular_movement(PI/4, 4)
sleep(1)

gyro_data = gyro.read_buffer()  # Le buffer contient seulement les 6 dernières secondes.
gyro_data = [x[2] for x in gyro_data]

rotation = 0.0
for reading in gyro_data:
    rotation += (reading - bias) * (1.0 / gyro.SAMPLE_RATE)
    
print('Rotation (rad): {}, Rotation (deg): {}'.format(rotation * 2 * PI / 360, rotation))

Rotation (rad): 2.159382087364927, Rotation (deg): 123.72347996216033


## Partie 3 - Variance sur le déplacement

Une autre conséquence de ces contrôleurs imparfaits, c'est que le déplacement du robot ne sera pas tout à fait constant pour une commande donnée. Il est habituel de modéliser ce phénomène avec une distribution gaussienne. On ne s'attend jamais à ce que le robot se déplace de $x$ radians, on s'attend plutôt qu'il se déplace en moyenne de $x$ radians avec une certaine variance.

Codez une boucle qui demande au robot de faire une grande quantité (autour de 20) de petites rotations. À chaque rotation, calculez le déplacement angulaire du point de vue de l'odomètre. Finalement, calculez l'écart type de la distribution ainsi calculée. La fonction `np.var` calcule la variance d'une liste.

In [53]:
def angle_from_odom_values(before, after):
    displacement_left = (after[1] - before[1]) * odometer.TICK_TO_METER
    displacement_right = (after[2] - before[2]) * odometer.TICK_TO_METER
    
    return (displacement_right - displacement_left) / (2 * robot.DISTANCE_CENTER_TO_WHEEL)

odom_measurements = []

for _ in range(20):
    odom_before = odometer.peek_data()
    robot.angular_movement(PI / 1.5, 0.8)
    sleep(1)
    odom_after = odometer.peek_data()
    odom_measurements.append(angle_from_odom_values(odom_before, odom_after))

In [54]:
odom_measurements = np.array(odom_measurements)
print('Measurements: ' + str(odom_measurements))
print('Standard deviation (rad): {}'.format(np.sqrt(np.var(odom_measurements))))

Measurements: [ 1.19292619  1.19525234  1.18943697  1.19408927  1.19602772  1.19331388
  1.19757849  1.19602772  1.19408927  1.19331388  1.18749851  1.20029233
  1.23247071  1.1615232   1.19680311  1.19486465  1.19835387  1.19835387
  1.19525234  1.1971908 ]
Standard deviation (rad): 0.011597398833699804
